# Fedbiomed Researcher

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Setting the client up

Install the celeba dataset with the help of the readme.md file inside `notebooks/data_for_examples/Celeba`  
For the sake of testing the resulting model, this file uses the data from node 1 and 2 for training and the data from node 3 to test.
you can create multiple nodes by adding a config parameter to the command contriling nodes, for example :  
creating 2 nodes for training :  
 - `./scripts/fedbiomed_run node config node1.ini start`
 - `./scripts/fedbiomed_run node config node2.ini start`  
 
adding data for each node :  
 - `./scripts/fedbiomed_run node config node1.ini add`
 - `./scripts/fedbiomed_run node config node2.ini add`

It is necessary to previously configure at least a node:
1. `./scripts/fedbiomed_run node config (ini file) add`
  * Select option 3 (images) to add an image dataset to the node
  * Add a name and the tag for the dataset (tag should contain '#celeba' as it is the tag used for this training) and finaly add the description
  * Pick a data folder from the 3 generated inside data/Celeba/celeba_preprocessed
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node config (ini file) list`
3. Run the node using `./scripts/fedbiomed_run node start`. Wait until you get `Connected with result code 0`. it means you are online.

for the sake of testing the resulting model, only nodes 1 and 2 were started during training, datas from node 3 is used to test the model.

## Create an experiment to train a model on the data found

Declare a torch.nn Net class to send for training on the node

In [2]:
from fedbiomed.researcher.environ import TMP_DIR
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=TMP_DIR+'/')
model_file = tmp_dir_model.name + '/CelebaClass.py'

Note : write **only** the code to export in the following cell

In [3]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from   fedbiomed.common.torchnn import TorchTrainingPlan
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image
import os


class Net(TorchTrainingPlan):
    def __init__(self):
        super(Net, self).__init__()
        #convolution layers
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 32, 3, 1)
        self.conv4 = nn.Conv2d(32, 32, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        # classifier
        self.fc1 = nn.Linear(3168, 128)
        self.fc2 = nn.Linear(128, 2)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        deps = ["from torch.utils.data import Dataset, DataLoader",
                "from torchvision import transforms",
                "import pandas as pd",
               "from PIL import Image",
               "import os",
               "import numpy as np"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)

        x = self.conv4(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


    class CelebaDataset(Dataset):
        """Custom Dataset for loading CelebA face images"""
        
        # we dont load the full data of the images, we retrieve the image with the get item. 
        # in our case, each image is 218*178 * 3colors. there is 67533 images. this take at leas 7G of ram
        # loading images when needed takes more time during training but it wont impact the ram usage as much as loading everything
        def __init__(self, txt_path, img_dir, transform=None):
            df = pd.read_csv(txt_path, sep="\t", index_col=0)
            self.img_dir = img_dir
            self.txt_path = txt_path
            self.img_names = df.index.values
            self.y = df['Smiling'].values
            self.transform = transform
            print("celeba dataset finished")

        def __getitem__(self, index):
            img = np.asarray(Image.open(os.path.join(self.img_dir,
                                        self.img_names[index])))
            img = transforms.ToTensor()(img)
            label = self.y[index]
            return img, label

        def __len__(self):
            return self.y.shape[0]
    
    def training_data(self,  batch_size = 48):
    # The training_data creates the Dataloader to be used for training in the general class Torchnn of fedbiomed
        dataset = self.CelebaDataset(self.dataset_path + "/target.csv", self.dataset_path + "/data/")
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = DataLoader(dataset, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        #this function must return the loss to backward it 
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


Writing /home/administrator/fedbiomed/fedbiomed/var/tmp/tmp6zb5l5fl/CelebaClass.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the client side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the client side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [4]:
training_args = {
    'batch_size': 32, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 200 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

# Train the federated model

In [5]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#celeba']
rounds = 5

exp = Experiment(tags=tags,
                 model_path=model_file,
                 model_class='Net',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 client_selection_strategy=None)

Messaging e7e0a833-c32c-4e5f-9624-4cde3c436cce connected with result code 0
Searching for clients with data tags: ['#celeba'] ...
2021-09-01 16:52:38.695029 [ RESEARCHER ] message received. {'researcher_id': 'researcher_4f6f3259-98d3-4ae0-959b-0ff8e79a9a0a', 'success': True, 'databases': [{'name': 'celeba', 'data_type': 'images', 'tags': ['#celeba'], 'description': 'celeba node 1', 'shape': [67533, 3, 218, 178], 'dataset_id': 'dataset_83c312fc-4a61-497c-8a71-002cf42fc986'}], 'count': 1, 'client_id': 'client_ce0fc899-2f1a-4006-ba7a-5d35f53231ea', 'command': 'search'}
2021-09-01 16:52:38.695757 [ RESEARCHER ] message received. {'researcher_id': 'researcher_4f6f3259-98d3-4ae0-959b-0ff8e79a9a0a', 'success': True, 'databases': [{'name': 'celeba', 'data_type': 'images', 'tags': ['#celeba'], 'description': 'celeba node 2', 'shape': [67533, 3, 218, 178], 'dataset_id': 'dataset_0552948d-e128-4d12-a024-c91000616a7d'}], 'count': 1, 'client_id': 'client_f89763a8-0d2e-4580-bf9e-e6ae3ddb7393', 'comm

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of client ID with `clients`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `client_selection_strategy` between the rounds

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [6]:
exp.run()

Sampled clients in round  0   ['client_ce0fc899-2f1a-4006-ba7a-5d35f53231ea', 'client_f89763a8-0d2e-4580-bf9e-e6ae3ddb7393']
[ RESEARCHER ] Send message to client  client_ce0fc899-2f1a-4006-ba7a-5d35f53231ea {'researcher_id': 'researcher_4f6f3259-98d3-4ae0-959b-0ff8e79a9a0a', 'job_id': '2337763c-8df9-4778-ac52-5f5bf133eb59', 'training_args': {'batch_size': 32, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 200}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/09/01/my_model_6dd80699-25f0-4090-849a-ad9093d9d9f9.py', 'params_url': 'http://localhost:8844/media/uploads/2021/09/01/my_model_a31a5597-0644-4aa9-8fbf-4fb29e4f6b8a.pt', 'model_class': 'Net', 'training_data': {'client_ce0fc899-2f1a-4006-ba7a-5d35f53231ea': ['dataset_83c312fc-4a61-497c-8a71-002cf42fc986']}}
researcher_4f6f3259-98d3-4ae0-959b-0ff8e79a9a0a
[ RESEARCHER ] Send message to client  client_f89763a8-0d2e-4580-bf9e-e6ae3ddb7393 {'researcher_id': 'researcher_4f6f3259-

2021-09-01 16:58:11.615398 [ RESEARCHER ] message received. {'researcher_id': 'researcher_4f6f3259-98d3-4ae0-959b-0ff8e79a9a0a', 'job_id': '2337763c-8df9-4778-ac52-5f5bf133eb59', 'success': True, 'client_id': 'client_ce0fc899-2f1a-4006-ba7a-5d35f53231ea', 'dataset_id': 'dataset_83c312fc-4a61-497c-8a71-002cf42fc986', 'params_url': 'http://localhost:8844/media/uploads/2021/09/01/node_params_709b5002-1859-40de-8888-7e56071486cb.pt', 'timing': {'rtime_training': 102.11056908500177, 'ptime_training': 589.05831949}, 'msg': '', 'command': 'train'}
2021-09-01 16:58:12.364039 [ RESEARCHER ] message received. {'researcher_id': 'researcher_4f6f3259-98d3-4ae0-959b-0ff8e79a9a0a', 'job_id': '2337763c-8df9-4778-ac52-5f5bf133eb59', 'success': True, 'client_id': 'client_f89763a8-0d2e-4580-bf9e-e6ae3ddb7393', 'dataset_id': 'dataset_0552948d-e128-4d12-a024-c91000616a7d', 'params_url': 'http://localhost:8844/media/uploads/2021/09/01/node_params_64f561ea-5137-479c-8750-5415d284e790.pt', 'timing': {'rtime_t

Retrieve the federated model parameters

In [7]:
fed_model = exp.model_instance
fed_model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

<All keys matched successfully>

In [8]:
fed_model

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=3168, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

# Test Model

We define a little testing routine to extract the accuracy metrics on the testing dataset
## Important
this is done to test the model because it is a devellopement environement  
in production, the data wont be accessible.

In [9]:

import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image
import os

def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'

    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    
        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/len(data_loader.dataset)

    return(test_loss, accuracy)

the test dataset is the data from the third node

In [10]:

test_dataset_path = "./data_for_examples/Celeba/celeba_preprocessed/data_node_3"

class CelebaDataset(Dataset):
        """Custom Dataset for loading CelebA face images"""

        def __init__(self, txt_path, img_dir, transform=None):
            df = pd.read_csv(txt_path, sep="\t", index_col=0)
            self.img_dir = img_dir
            self.txt_path = txt_path
            self.img_names = df.index.values
            self.y = df['Smiling'].values
            self.transform = transform
            print("celeba dataset finished")

        def __getitem__(self, index):
            img = np.asarray(Image.open(os.path.join(self.img_dir,
                                        self.img_names[index])))
            img = transforms.ToTensor()(img)
            label = self.y[index]
            return img, label

        def __len__(self):
            return self.y.shape[0]
    

dataset = CelebaDataset(test_dataset_path + "/target.csv", test_dataset_path + "/data/")
train_kwargs = {'batch_size': 64, 'shuffle': True}
data_loader = DataLoader(dataset, **train_kwargs)

celeba dataset finished


Loading the testing dataset and computing accuracy metrics for local and federated models

In [11]:
acc_federated = testing_Accuracy(fed_model, data_loader)

In [12]:
acc_federated[1]

88.29905379592199